In [1]:
# Import required libraries
# Add whatever you want
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Test tensorflow
print('TensorFlow version: ' + tf.__version__)
a = tf.constant(2.0)
b = tf.constant(3.0)
c = a * b

sess = tf.Session()
result = sess.run([a, b, c])
print('%f * %f = %f' % (result[0], result[1], result[2]))
sess.close()

# load data
TRAIN_FOLDER = '../data/normalized/train'
TEST_FOLDER = '../data/normalized/test'
ID_FILE = '../data/normalized/id.json'
NUM_SPEAKERS = 200

def get_attributes(fname):
    attr = fname.split('.')[0].split('-')
    dialect = attr[0]
    gender = attr[1][0]
    speaker_id = attr[1]
    sentence_type = attr[2][:2]
    return dialect, gender, speaker_id, sentence_type

def load_files(train):
    X = []
    Y = []
    for speaker_id, flist in train.items():
        for fname in flist:
            filedata = np.load(fname)
            for segment in filedata:
                for x in segment:
                    X.append(x) # dim(x) => (390,)
                    Y.append(speaker_id)
    return np.array(X).astype(np.float32), np.array(Y)
    

def load_train_data():
    import json
    with open(ID_FILE, 'r') as f_json:
        ids = json.load(f_json)
    print('ids', ids)
    
    
    files = [f for f in os.listdir(TRAIN_FOLDER) if os.path.isfile(os.path.join(TRAIN_FOLDER, f))]

    # split train val data
    train = {}
    val = {}
    
    added = {}
    for file in files:
        dialect, gender, speaker_id, sentence_type = get_attributes(file)
        file_path = os.path.join(TRAIN_FOLDER, file)
        if speaker_id not in ids:
            print('ERROR:', speaker_id, 'not found in ids')
        speaker_id = ids[speaker_id]
        val_set = added.setdefault(speaker_id, {})
        if sentence_type not in val_set:
            val.setdefault(speaker_id, []).append(file_path)
            val_set[sentence_type] = True
        else:
            train.setdefault(speaker_id, []).append(file_path)
    # test data    
    test = {}
    files = [f for f in os.listdir(TEST_FOLDER) if os.path.isfile(os.path.join(TEST_FOLDER, f))]
    for file in files:
        dialect, gender, speaker_id, sentence_type = get_attributes(file)
        file_path = os.path.join(TEST_FOLDER, file)
        if speaker_id not in ids:
            print('ERROR:', speaker_id, 'not found in ids')
        speaker_id = ids[speaker_id]
        test.setdefault(speaker_id, []).append(file_path)
    
    # load data
    # input_path = os.path.join(TRAIN_FOLDER, file)
    X_train, Y_train = load_files(train)
    X_val, Y_val = load_files(val)
    X_test, Y_test = load_files(test)
    return X_train, Y_train, X_val, Y_val, X_test, Y_test
    
print('Loading data...')
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_train_data()

print()
print('# Train :', len(Y_train))
print('# Eval  :', len(Y_val))
print('# Test  :', len(Y_test))
print()

TensorFlow version: 1.3.1
2.000000 * 3.000000 = 6.000000
Loading data...
ids {'MJDG0': 111, 'MDPB0': 60, 'MJRP0': 136, 'MEWM0': 78, 'MILB0': 103, 'MDWD0': 67, 'MJAI0': 106, 'MDLR0': 56, 'MJMA0': 125, 'MDLC2': 53, 'MKLR0': 151, 'MMWS0': 184, 'MJJB0': 119, 'MKAG0': 143, 'MARW0': 11, 'MCEW0': 28, 'MDSS0': 64, 'MAEO0': 4, 'MMDM1': 170, 'MKLS1': 152, 'MDSS1': 65, 'MJBG0': 107, 'MMDS0': 171, 'MFMC0': 80, 'MJJJ0': 120, 'MBOM0': 20, 'MPFU0': 191, 'MJAE0': 105, 'MDLC0': 51, 'MJJM0': 121, 'MJKR0': 122, 'MMWS1': 185, 'MFER0': 79, 'MDAS0': 38, 'MFRM0': 81, 'MEDR0': 71, 'MDLH0': 54, 'MGES0': 90, 'MCTH0': 35, 'MJWT0': 140, 'MGAR0': 88, 'MFXS0': 83, 'MHJB0': 98, 'MJWG0': 138, 'MHRM0': 101, 'MCSS0': 34, 'MCDD0': 25, 'MJEB0': 113, 'MDWM0': 69, 'MJDE0': 110, 'MKJO0': 150, 'MPPC0': 196, 'MJXL0': 142, 'MMEB0': 173, 'MDSJ0': 63, 'MDDC0': 43, 'MKAJ0': 145, 'MAJP0': 6, 'MBMA1': 18, 'MDTB0': 66, 'MJPG0': 128, 'MDLM0': 55, 'MDLB0': 50, 'MMAG0': 163, 'MDEF0': 45, 'MPGH0': 192, 'MDBP0': 40, 'MDEM0': 46, 'MCLK0':

In [2]:
input_nodes = 390
hidden_nodes = 200
output_nodes = NUM_SPEAKERS
batch_size = 128
beta = 0.0001 # lambda for regularization

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, [None, 390])
    labels = tf.placeholder(tf.int64, [None])
    tf_train_labels = tf.one_hot(labels, output_nodes)
    tf_valid_dataset = tf.constant(X_val)
    tf_test_dataset = tf.constant(X_test)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([input_nodes, hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, output_nodes]))
    biases_2 = tf.Variable(tf.zeros([output_nodes]))

    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    sigmoid_layer = tf.nn.sigmoid(logits_1)
    logits_2 = tf.matmul(sigmoid_layer, weights_2) + biases_2
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    # Loss function with L2 Regularization with beta=0.01
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 1e-4

    # tf.train.exponential_decay(learning_rate, global_step, decay_steps, 
    # decay_rate, staircase=False, name=None)
    # staircase: Boolean. If True decay the learning rate at discrete intervals

    # decay every 500 steps with a base of 0.96
    learning_rate = tf.train.exponential_decay(starter_learning_rate,
                                                    global_step, 500, 0.96,
                                                    staircase=True)

    # Passing global_step to minimize() will increment it at each step.
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    sigmoid_layer = tf.nn.sigmoid(logits_1)
    logits_2 = tf.matmul(sigmoid_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    sigmoid_layer = tf.nn.sigmoid(logits_1)
    logits_2 = tf.matmul(sigmoid_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

In [5]:
num_epoch = 40
log_step = 50

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == labels)
            / predictions.shape[0])

from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train, Y_train)

train_dataset = X_train
train_labels = Y_train
valid_labels = Y_val
test_labels = Y_test

curr_step = 0
num_training = len(train_dataset)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for epoch in range(num_epoch):
        print('===== > train for epoch %d' % epoch)
        for step in range(num_training // batch_size):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size)]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            curr_step += 1
            if (curr_step % log_step == 0):
                print('iteration (%d): loss = %.3f, accuracy = %.3f' %
                        (curr_step, l, accuracy(predictions, batch_labels)))
        print('-  epoch %d: validation accuracy = %.3f' % (epoch, accuracy(valid_prediction.eval(), valid_labels)))
        print()
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
===== > train for epoch 0
iteration (50): loss = 476.294, accuracy = 0.000
iteration (100): loss = 471.762, accuracy = 0.781
iteration (150): loss = 466.807, accuracy = 0.781
iteration (200): loss = 461.129, accuracy = 1.562
iteration (250): loss = 458.204, accuracy = 0.000
iteration (300): loss = 453.277, accuracy = 1.562
iteration (350): loss = 450.058, accuracy = 0.000
iteration (400): loss = 445.014, accuracy = 0.000
iteration (450): loss = 441.025, accuracy = 0.000
iteration (500): loss = 437.211, accuracy = 0.000
-  epoch 0: validation accuracy = 0.516

===== > train for epoch 1
iteration (550): loss = 433.528, accuracy = 0.781
iteration (600): loss = 429.964, accuracy = 0.781
iteration (650): loss = 424.921, accuracy = 1.562
iteration (700): loss = 421.877, accuracy = 1.562
iteration (750): loss = 418.707, accuracy = 0.000
iteration (800): loss = 414.090, accuracy = 0.000
iteration (850): loss = 410.261, accuracy = 0.000
iteration (900): loss = 405.389, accuracy = 1.